In [1]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import Sequential, Model
from tensorflow.keras.preprocessing.sequence import pad_sequences
from pprint import pprint

print(tf.__version__)


sentences = [
     ['I', 'feel', 'hungry'],
     ['tensorflow', 'is', 'very', 'difficult'],
     ['tensorflow', 'is', 'a', 'framework', 'for', 'deep', 'learning'],
     ['tensorflow', 'is', 'very', 'fast', 'changing']]

pos = [
    ['pronoun', 'verb', 'adjective'],
    ['noun', 'verb', 'adverb', 'adjective'],
    ['noun', 'verb', 'determiner', 'noun', 'preposition', 'adjective', 'noun'],
    ['noun', 'verb', 'adverb', 'adjective', 'verb']]


word_list = np.sum(sentences)
word_list = sorted(set(word_list))
word_list = ['<pad>'] + word_list
word2idx = {word : idx for idx, word in enumerate(word_list)}
idx2word = {idx : word for idx, word in enumerate(word_list)}

print(word_list)
print(word2idx)
print(idx2word)
print(len(idx2word))


pos_list = np.sum(pos)
pos_list = sorted(set(pos_list))
pos_list = ['<pad>'] + pos_list
pos2idx = {pos : idx for idx, pos in enumerate(pos_list)}
idx2pos = {idx : pos for idx, pos in enumerate(pos_list)}

print(pos_list)
print(pos2idx)
print(idx2pos)
print(len(pos2idx))



max_sequence = 10
x_data = list(map(lambda sentence : [word2idx.get(token) for token in sentence], sentences))
y_data = list(map(lambda sentence : [pos2idx.get(token) for token in sentence], pos))


x_data
y_data

# padding the sequence of indices
x_data = pad_sequences(sequences = x_data, maxlen = max_sequence, padding='post')
x_data_mask = ((x_data != 0) * 1).astype(np.float32)
x_data_len = list(map(lambda sentence : len(sentence), sentences))

y_data = pad_sequences(sequences = y_data, maxlen = max_sequence, padding='post')

# checking data
print(x_data, x_data_len)
print(x_data_mask)
print(y_data)
print(y_data.shape)



num_classes = len(pos2idx)
hidden_dim = [10, 10]

input_dim = len(word2idx)
output_dim = len(word2idx)
one_hot = np.eye(len(word2idx))

1.14.0-rc1
['<pad>', 'I', 'a', 'changing', 'deep', 'difficult', 'fast', 'feel', 'for', 'framework', 'hungry', 'is', 'learning', 'tensorflow', 'very']
{'<pad>': 0, 'I': 1, 'a': 2, 'changing': 3, 'deep': 4, 'difficult': 5, 'fast': 6, 'feel': 7, 'for': 8, 'framework': 9, 'hungry': 10, 'is': 11, 'learning': 12, 'tensorflow': 13, 'very': 14}
{0: '<pad>', 1: 'I', 2: 'a', 3: 'changing', 4: 'deep', 5: 'difficult', 6: 'fast', 7: 'feel', 8: 'for', 9: 'framework', 10: 'hungry', 11: 'is', 12: 'learning', 13: 'tensorflow', 14: 'very'}
15
['<pad>', 'adjective', 'adverb', 'determiner', 'noun', 'preposition', 'pronoun', 'verb']
{'<pad>': 0, 'adjective': 1, 'adverb': 2, 'determiner': 3, 'noun': 4, 'preposition': 5, 'pronoun': 6, 'verb': 7}
{0: '<pad>', 1: 'adjective', 2: 'adverb', 3: 'determiner', 4: 'noun', 5: 'preposition', 6: 'pronoun', 7: 'verb'}
8
[[ 1  7 10  0  0  0  0  0  0  0]
 [13 11 14  5  0  0  0  0  0  0]
 [13 11  2  9  8  4 12  0  0  0]
 [13 11 14  6  3  0  0  0  0  0]] [3, 4, 7, 5]
[[1. 1

In [0]:
def create_model():
  model = Sequential()
  model.add(layers.Embedding(input_dim=input_dim, output_dim=output_dim, mask_zero=True,
                             trainable=False, input_length=max_sequence,
                             embeddings_initializer=keras.initializers.Constant(one_hot)))
  model.add(layers.SimpleRNN(units=hidden_dim[0], return_sequences=True))
  model.add(layers.TimeDistributed(layers.Dropout(rate=0.3)))

  model.add(layers.SimpleRNN(units=hidden_dim[1], return_sequences=True))
  model.add(layers.Dropout(rate=0.3))
  model.add(layers.Dense(units=num_classes))
  
  return model

In [0]:
def create_bidirectional_model():
  model = Sequential()
  model.add(layers.InputLayer(input_shape=(max_sequence, )))
  model.add(layers.Embedding(input_dim=input_dim, output_dim=output_dim, mask_zero=True,
                             trainable=False, input_length=max_sequence,
                             embeddings_initializer=keras.initializers.Constant(one_hot)))
  model.add(layers.Bidirectional(keras.layers.SimpleRNN(units=hidden_dim[0], return_sequences=True)))
  model.add(layers.TimeDistributed(layers.Dense(units=num_classes)))
  
  return model

In [0]:
model = create_model()

In [3]:
model = create_bidirectional_model()

W0627 11:17:29.224587 140561740134272 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0627 11:17:29.233208 140561740134272 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/init_ops.py:97: calling GlorotUniform.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0627 11:17:29.235073 140561740134272 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/init_ops.py:97: calling Orthogonal.__init__ (from tensorflow.python.ops.init_ops) with

In [4]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 10, 15)            225       
_________________________________________________________________
bidirectional (Bidirectional (None, 10, 20)            520       
_________________________________________________________________
time_distributed (TimeDistri (None, 10, 8)             168       
Total params: 913
Trainable params: 688
Non-trainable params: 225
_________________________________________________________________


In [0]:
def loss_fn(model, x, y, x_len, max_sequence):
    masking = tf.sequence_mask(x_len, maxlen=max_sequence, dtype=tf.float32)
    valid_time_step = tf.cast(x_len,dtype=tf.float32)
    sequence_loss = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=model(x)) * masking
    sequence_loss = tf.reduce_sum(sequence_loss, axis=-1) / valid_time_step
    sequence_loss = tf.reduce_mean(sequence_loss)
    return sequence_loss

In [6]:
# creating and optimizer
lr = 0.1
epochs = 30
batch_size = 2
opt = tf.keras.optimizers.Adam(learning_rate = lr)


tr_dataset = tf.data.Dataset.from_tensor_slices((x_data, y_data, x_data_len))
tr_dataset = tr_dataset.shuffle(buffer_size=4)
tr_dataset = tr_dataset.batch(batch_size = 2)

print(tr_dataset)

<DatasetV1Adapter shapes: ((?, 10), (?, 10), (?,)), types: (tf.int32, tf.int32, tf.int32)>


In [8]:
tr_loss_hist = []

for epoch in range(epochs):
    avg_tr_loss = 0
    tr_step = 0

    for x_mb, y_mb, x_mb_len in tr_dataset:
        with tf.GradientTape() as tape:
            tr_loss = loss_fn(model, x=x_mb, y=y_mb, x_len=x_mb_len, max_sequence=max_sequence)
        grads = tape.gradient(target=tr_loss, sources=model.trainable_variables)
        opt.apply_gradients(grads_and_vars=zip(grads, model.trainable_variables))
        avg_tr_loss += tr_loss
        tr_step += 1
    else:
        avg_tr_loss /= tr_step
        tr_loss_hist.append(avg_tr_loss)

    if (epoch + 1) % 5 == 0:
        print('epoch : {:3}, tr_loss : {:.3f}'.format(epoch + 1, avg_tr_loss))


yhat = model.predict(x_data)
yhat = np.argmax(yhat, axis=-1) * x_data_mask

print(yhat)

pprint(list(map(lambda row : [idx2pos.get(elm) for elm in row],yhat.astype(np.int32).tolist())), width = 120)
pprint(pos)

plt.plot(tr_loss_hist)
plt.show()

W0627 11:25:03.175654 140561740134272 optimizer_v2.py:979] Gradients does not exist for variables ['bidirectional/forward_simple_rnn/kernel:0', 'bidirectional/forward_simple_rnn/recurrent_kernel:0', 'bidirectional/forward_simple_rnn/bias:0', 'bidirectional/backward_simple_rnn/kernel:0', 'bidirectional/backward_simple_rnn/recurrent_kernel:0', 'bidirectional/backward_simple_rnn/bias:0'] when minimizing the loss.
W0627 11:25:03.463026 140561740134272 optimizer_v2.py:979] Gradients does not exist for variables ['bidirectional/forward_simple_rnn/kernel:0', 'bidirectional/forward_simple_rnn/recurrent_kernel:0', 'bidirectional/forward_simple_rnn/bias:0', 'bidirectional/backward_simple_rnn/kernel:0', 'bidirectional/backward_simple_rnn/recurrent_kernel:0', 'bidirectional/backward_simple_rnn/bias:0'] when minimizing the loss.
W0627 11:25:03.742258 140561740134272 optimizer_v2.py:979] Gradients does not exist for variables ['bidirectional/forward_simple_rnn/kernel:0', 'bidirectional/forward_simpl

KeyboardInterrupt: ignored